In [1]:
import json
with open("all_v1.json", "r") as file:
    data = json.load(file)

In [5]:
text = """PRIVACY POLICY
This Data Protection Notice ("Notice") sets out the basis which Hackerschool Pte Ltd ("we", "us", or "our") may collect, use, disclose or otherwise process personal data in accordance with the Personal Data Protection Act ("PDPA"). This Notice applies to personal data in our possession or under our control, including personal data in the possession of organisations which we have engaged to collect, use, disclose or process personal data for our purposes.

PERSONAL DATA
As used in this Notice:

"personal data" means data, whether true or not, about a person who can be identified: (a) from that data; or (b) from that data and other information to which we have or are likely to have access.

Depending on the nature of your interaction with us, some examples of personal data which we may collect from you include your name and identification information such as your email address, telephone number, and gender.
Other terms used in this Notice shall have the meanings given to them in the PDPA (where the context so permits).
COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA
We generally do not collect your personal data unless (a) it is provided to us voluntarily by you directly or via a third party who has been duly authorised by you to disclose your personal data to us (your "authorised representative") after (i) you (or your authorised representative) have been notified of the purposes for which the data is collected, and (ii) you (or your authorised representative) have provided written consent to the collection and usage of your personal data for those purposes, or (b) collection and use of personal data without consent is permitted or required by the PDPA or other laws. We shall seek your consent before collecting any additional personal data and before using your personal data for a purpose which has not been notified to you (except where permitted or authorised by law).
We may collect and use your personal data for any or all of the following purposes: (a) to notify you of updates, or contact you regarding an event you have signed up for; and (b) to administer your participation in an event you have signed up for.
We may disclose your personal data: (a) where such disclosure is required for performing obligations in the course of or in connection with our provision of the goods or services requested by you; or (b) to third party service providers, agents and other organisations we have engaged to perform any of the functions listed in clause 5 above for us.
The purposes listed in the above clauses may continue to apply even in situations where your relationship with us (for example, pursuant to a contract) has been terminated or altered in any way, for a reasonable period thereafter (including, where applicable, a period to enable us to enforce our rights under any contract with you).
WITHDRAWING YOUR CONSENT
The consent that you provide for the collection, use and disclosure of your personal data will remain valid until such time it is being withdrawn by you in writing. You may withdraw consent and request us to stop using and/or disclosing your personal data for any or all of the purposes listed above by submitting your request in writing or via email to our Data Protection Officer at the contact details provided below.
Upon receipt of your written request to withdraw your consent, we may require reasonable time (depending on the complexity of the request and its impact on our relationship with you) for your request to be processed and for us to notify you of the consequences of us acceding to the same, including any legal consequences which may affect your rights and liabilities to us. In general, we shall seek to process your request within ten (10) business days of receiving it.
Whilst we respect your decision to withdraw your consent, please note that depending on the nature and scope of your request, we may not be in a position to continue providing our goods or services to you and we shall, in such circumstances, notify you before completing the processing of your request. Should you decide to cancel your withdrawal of consent, please inform us in writing in the manner described in clause 8 above.
Please note that withdrawing consent does not affect our right to continue to collect, use and disclose personal data where such collection, use and disclose without consent is permitted or required under applicable laws.
ACCESS TO AND CORRECTION OF PERSONAL DATA
If you wish to make (a) an access request for access to a copy of the personal data which we hold about you or information about the ways in which we use or disclose your personal data, or (b) a correction request to correct or update any of your personal data which we hold about you, you may submit your request in writing or via email to our Data Protection Officer at the contact details provided below.
Please note that a reasonable fee may be charged for an access request. If so, we will inform you of the fee before processing your request.
We will respond to your request as soon as reasonably possible. Should we not be able to respond to your request within thirty (30) days after receiving your request, we will inform you in writing within thirty (30) days of the time by which we will be able to respond to your request. If we are unable to provide you with any personal data or to make a correction requested by you, we shall generally inform you of the reasons why we are unable to do so (except where we are not required to do so under the PDPA).
PROTECTION OF PERSONAL DATA
To safeguard your personal data from unauthorised access, collection, use, disclosure, copying, modification, disposal or similar risks, we have introduced appropriate administrative, physical and technical measures such as up-to-date antivirus protection, encryption and the use of privacy filters to secure all storage and transmission of personal data by us, and disclosing personal data both internally and to our authorised third party service providers and agents only on a need-to-know basis.
You should be aware, however, that no method of transmission over the Internet or method of electronic storage is completely secure. While security cannot be guaranteed, we strive to protect the security of your information and are constantly reviewing and enhancing our information security measures.
ACCURACY OF PERSONAL DATA
We generally rely on personal data provided by you (or your authorised representative). In order to ensure that your personal data is current, complete and accurate, please update us if there are changes to your personal data by informing our Data Protection Officer in writing or via email at the contact details provided below.
RETENTION OF PERSONAL DATA
We may retain your personal data for as long as it is necessary to fulfil the purpose for which it was collected, or as required or permitted by applicable laws.
We will cease to retain your personal data, or remove the means by which the data can be associated with you, as soon as it is reasonable to assume that such retention no longer serves the purpose for which the personal data was collected, and is no longer necessary for legal or business purposes.
TRANSFERS OF PERSONAL DATA OUTSIDE OF SINGAPORE
We generally do not transfer your personal data to countries outside of Singapore. However, if we do so, we will obtain your consent for the transfer to be made and we will take steps to ensure that your personal data continues to receive a standard of protection that is at least comparable to that provided under the PDPA.
DATA PROTECTION OFFICER
You may contact our Data Protection Officer if you have any enquiries or feedback on our personal data protection policies and procedures, or if you wish to make any request, by email at pdpa@nushackers.org.
EFFECT OF NOTICE AND CHANGES TO NOTICE
This Notice applies in conjunction with any other notices, contractual clauses and consent clauses that apply in relation to the collection, use and disclosure of your personal data by us.
We may revise this Notice from time to time without any prior notice. You may determine if any such revision has taken place by referring to the date on which this Notice was last updated. Your continued use of our services constitutes your acknowledgement and acceptance of such changes."""

In [8]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [10]:
doc = nlp(text)

for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_)

PRIVACY PROPN NNP compound
POLICY PROPN NNP nsubj

 SPACE _SP 
This DET DT det
Data PROPN NNPS compound
Protection PROPN NNP compound
Notice PROPN NNP appos
( PUNCT -LRB- punct
" PUNCT `` punct
Notice PROPN NNP appos
" PUNCT '' punct
) PUNCT -RRB- punct
sets VERB VBZ ROOT
out ADP RP prt
the DET DT det
basis NOUN NN dobj
which PRON WDT dobj
Hackerschool PROPN NNP compound
Pte PROPN NNP compound
Ltd PROPN NNP nsubj
( PUNCT -LRB- punct
" PUNCT `` punct
we PRON PRP nsubj
" PUNCT '' punct
, PUNCT , punct
" PUNCT '' punct
us PRON PRP conj
" PUNCT '' punct
, PUNCT , punct
or CCONJ CC cc
" PUNCT `` punct
our PRON PRP$ conj
" PUNCT '' punct
) PUNCT -RRB- punct
may AUX MD aux
collect VERB VB relcl
, PUNCT , punct
use VERB VB conj
, PUNCT , punct
disclose VERB VBP conj
or CCONJ CC cc
otherwise ADV RB advmod
process VERB VB conj
personal ADJ JJ amod
data NOUN NNS dobj
in ADP IN prep
accordance NOUN NN pobj
with ADP IN prep
the DET DT det
Personal PROPN NNP compound
Data PROPN NNPS compound
Protect

laws NOUN NNS pobj
. PUNCT . punct

 SPACE _SP 
ACCESS PROPN NNP ROOT
TO ADP IN prep
AND CCONJ CC cc
CORRECTION NOUN NN conj
OF ADP IN prep
PERSONAL ADJ JJ compound
DATA NOUN NNS pobj

 SPACE _SP 
If SCONJ IN mark
you PRON PRP nsubj
wish VERB VBP advcl
to PART TO aux
make VERB VB xcomp
( PUNCT -LRB- punct
a DET DT meta
) PUNCT -RRB- punct
an DET DT det
access NOUN NN compound
request NOUN NN dobj
for ADP IN prep
access NOUN NN pobj
to ADP IN prep
a DET DT det
copy NOUN NN pobj
of ADP IN prep
the DET DT det
personal ADJ JJ amod
data NOUN NNS pobj
which PRON WDT dobj
we PRON PRP nsubj
hold VERB VBP relcl
about ADP IN prep
you PRON PRP pobj
or CCONJ CC cc
information NOUN NN conj
about ADP IN prep
the DET DT det
ways NOUN NNS pobj
in ADP IN prep
which PRON WDT pobj
we PRON PRP nsubj
use VERB VBP relcl
or CCONJ CC cc
disclose VERB VBP conj
your PRON PRP$ poss
personal ADJ JJ amod
data NOUN NNS dobj
, PUNCT , punct
or CCONJ CC cc
( PUNCT -LRB- punct
b X LS meta
) PUNCT -RRB- punct
a DET DT 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



cc
via ADP IN conj
email NOUN NN pobj
at ADP IN prep
the DET DT det
contact NOUN NN compound
details NOUN NNS pobj
provided VERB VBN acl
below ADV RB advmod
. PUNCT . punct

 SPACE _SP 
RETENTION PROPN NNP ROOT
OF ADP IN prep
PERSONAL ADJ JJ compound
DATA NOUN NNS pobj

 SPACE _SP 
We PRON PRP nsubj
may AUX MD aux
retain VERB VB ROOT
your PRON PRP$ poss
personal ADJ JJ amod
data NOUN NNS dobj
for ADP IN prep
as ADV RB advmod
long ADV RB pcomp
as SCONJ IN mark
it PRON PRP nsubj
is AUX VBZ advcl
necessary ADJ JJ acomp
to PART TO aux
fulfil VERB VB xcomp
the DET DT det
purpose NOUN NN dobj
for ADP IN prep
which PRON WDT pobj
it PRON PRP nsubjpass
was AUX VBD auxpass
collected VERB VBN relcl
, PUNCT , punct
or CCONJ CC cc
as ADV RB mark
required VERB VBN conj
or CCONJ CC cc
permitted VERB VBN conj
by ADP IN agent
applicable ADJ JJ amod
laws NOUN NNS pobj
. PUNCT . punct

 SPACE _SP 
We PRON PRP nsubj
will AUX MD aux
cease VERB VB ROOT
to PART TO aux
retain VERB VB xcomp
your PRON PRP$ poss

In [14]:
import requests
from bs4 import BeautifulSoup
r = requests.get("https://tosdr.org/index.html#")

In [69]:
soup = BeautifulSoup(r.text, "html.parser")
spans = soup.findAll("div", class_="span6")
data = []
classi = {
    "badge-warning": -1,
    "badge-neutral": 0,
    "badge-success": 1,
    "badge-important": -1
}
for span in spans:
    for item in span.findAll("li"):
        classification = item.span['class']
        classification.remove('badge')
        text = item.text[2:]
        data.append({
            "text": text,
            "rating": classi[classification[0]]
        })

In [76]:
with open("sentiment_data.json", "w") as file:
    json.dump(data, file)

In [127]:
headers = {
    'X-RapidAPI-Host': 'ipeirotis-readability-metrics.p.rapidapi.com',
    'X-RapidAPI-Key': 'f9ubYNvChSmsh7ziUqOiramFPcBnp1UxuM2jsnCqrPp4oiO7Dw'
}
r = requests.post("https://ipeirotis-readability-metrics.p.rapidapi.com/getReadabilityMetrics", headers=headers, data={"text":"test"})
response = r.json()
response

{'SYLLABLES': 1.0,
 'SENTENCES': 1.0,
 'SMOG_INDEX': 3.0,
 'FLESCH_READING': 121.22,
 'COLEMAN_LIAU': -22.24,
 'CHARACTERS': 4.0,
 'ARI': -2.09,
 'FLESCH_KINCAID': -3.4,
 'SMOG': 3.129,
 'WORDS': 1.0,
 'GUNNING_FOG': 0.4,
 'COMPLEXWORDS': 0.0}

In [139]:
import re
from sumy.nlp.stemmers import Stemmer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
import string
import unidecode
import json
from bs4 import BeautifulSoup
import requests
LANGUAGE = 'english'
SENTENCES_COUNT = 15
clause = re.compile("(will)|(agree)|(must)|(responsib)|(waive)|(lawsuit)|(modify)|(intellec)", re.IGNORECASE)
host_reg = re.compile('(?:http.*://)?(?P<host>[^:/ ]+).?(?P<port>[0-9]*).*')
terms_page = re.compile(r"(terms *(((and|&)? *conditions)|((of)? ?(service|use))))", re.IGNORECASE)
HTML_OPEN = "<div id='mainPopup'>"
YOU_AGREE_HEADER = "<div id='youAgree' class='header'>What You Agree</div>"
THEY_AGREE_HEADER = "<div id='theyAgree' class='header'>What They Agree</div>"
OTHER_HEADER = "<div id='other' class='header'>Other Clauses</div>"

def prepare_for_regex(input_str, delimiter="."):
    pure_sentences = input_str.strip().split(delimiter)
    if pure_sentences[-1] == "":
        pure_sentences.pop()

    # Remove leading/trailing whitespace and end with period.
    for i, x in enumerate(pure_sentences):
        pure_sentences[i] = x.strip() + "."

    clean_sentences = []
    for x in pure_sentences:
        clean_sentences.append(' '.join(x.lower().translate(str.maketrans({key: None for key in string.punctuation})).split()))
    return clean_sentences, pure_sentences

In [239]:
link = "https://hrtechprivacy.com"
r = requests.get(link)
html_soup = BeautifulSoup(r.text, "html.parser")
text = html_soup.text
        
text_data = unidecode.unidecode(text)
clean_list, pure_list = prepare_for_regex(text_data)

data_to_summarize = []
for clean, pure in zip(clean_list, pure_list):
    if re.findall(clause, clean):
        data_to_summarize.append(pure)
text_data = " ".join(data_to_summarize)
parser = PlaintextParser(text_data, Tokenizer(LANGUAGE))
stemmer = Stemmer(LANGUAGE)
summarizer = TextRankSummarizer(stemmer)

summary = summarizer(parser.document, SENTENCES_COUNT)

In [242]:
text

'\n403 Forbidden\n\n403 Forbidden\nnginx\n\n\n'

In [209]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
text = re.sub(cleanr, '', text_data)

In [210]:
sentences = [str(sentence) for sentence in summary]

In [236]:
from IPython.core.display import HTML,display
all_sentences = []
for sent in sentences:
    sent = nlp(sent)
    sentence = ""
    entities = []
    for token in sent:
        if sentence and token.text.strip() not in string.punctuation:
            sentence += " "
        if token.pos_ == "VERB":
            sentence += '''<mark class="entity" style="background: #ffffb3; padding: 0.2em 0.2em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone"> {}</mark>'''.format(token.text)
            entities.append(token.text)
        else:
            sentence += token.text
    sentence = sentence[:-1] + ". "
    for ent in sent.ents:
        if ent.text not in entities and ent.label_ == "ORG":
            sentence = sentence.replace(ent.text, '''<mark class="entity" style="background: #ccffcc; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone"> {} <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">ORG</span></mark> '''.format(ent.text))
            entities.append(ent.text)
    all_sentences.append(sentence)

In [237]:
display(HTML("<br><br>".join(all_sentences)))

In [184]:
from IPython.core.display import HTML,display
all_sentences = []
TERMS = ["withdraw", "provide", "remain"]
for sentence in sentences:
    for term in TERMS:
        case_insensitive = re.compile(re.escape(term), re.IGNORECASE)
        sentence = case_insensitive.sub('''<mark class="entity" style="background: #ffffb3; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone"> {} <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">VERB</span></mark>'''.format(term), sentence)
    all_sentences.append(sentence)

the Spotify Service
Agreements
Spotify Customer Service
Us


In [238]:
!pip install textblob

  Using cached https://files.pythonhosted.org/packages/60/f0/1d9bfcc8ee6b83472ec571406bd0dd51c0e6330ff1a51b2d29861d389e85/textblob-0.15.3-py2.py3-none-any.whl
